In [ ]:
!pip install torch transformers

In [ ]:
# Step 1: Import Required Libraries
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

In [ ]:
texts = ["I love PyTorch!", "I hate bugs in my code."]
labels = [1, 0]


In [ ]:
# Step 1: Import Required Libraries
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim

# Explanation:
# - `torch` is the main PyTorch library.
# - `transformers` is from Hugging Face, giving us pre-trained BERT and utilities like tokenizers.
# - `Dataset` and `DataLoader` are for handling input data efficiently.
# - `nn` and `optim` are used to define our model architecture and optimization.

# Step 2: Define Input Texts and Labels
texts = ["I love PyTorch!", "I hate bugs in my code."]
labels = [1, 0]  # 1 = Positive, 0 = Negative sentiment

# Explanation:
# - `texts` is a list of input sentences for the model.
# - `labels` are the corresponding ground-truth sentiments.

# Step 3: Tokenization
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Tokenize and convert to input IDs
tokens = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Explanation:
# - `BertTokenizer.from_pretrained("bert-base-uncased")` loads a tokenizer for the BERT model.
# - `padding=True`: Ensures all sentences are the same length by adding padding tokens.
# - `truncation=True`: Truncates long sentences to the max length BERT can handle (512 by default).
# - `return_tensors="pt"`: Converts the output to PyTorch tensors.

# Inputs after tokenization
input_ids = tokens["input_ids"]
attention_mask = tokens["attention_mask"]

# Explanation:
# - `input_ids`: Encoded representation of words as integers (vocab indices).
# - `attention_mask`: Binary tensor indicating which positions are actual words (1) or padding (0).

# Step 4: Define the Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Explanation:
# - `BertForSequenceClassification`: A pre-trained BERT model adapted for classification tasks.
# - `num_labels=2`: Indicates binary classification (positive/negative sentiment).

# Step 5: Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-5)

# Explanation:
# - `nn.CrossEntropyLoss()`: Common loss function for classification problems. It compares predicted probabilities with true labels.
# - `optim.AdamW`: Optimizer for updating model weights. `lr=1e-5` is the learning rate.

# Step 6: Training Loop
epochs = 2
batch_size = 2

# Create a Dataset and DataLoader for batching
class SentimentDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
            "labels": self.labels[idx],
        }

dataset = SentimentDataset(input_ids, attention_mask, torch.tensor(labels))
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Explanation:
# - `SentimentDataset`: Custom Dataset class to bundle input data for DataLoader.
# - `__getitem__`: Allows the DataLoader to retrieve individual samples as dictionaries.
# - `DataLoader`: Handles batching and shuffling for efficient training.

# Training the model
model.train()  # Set model to training mode

for epoch in range(epochs):
    for batch in dataloader:
        # Extract batch data
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Explanation:
        # - `model(input_ids, attention_mask)`: BERT processes the inputs.
        # - `labels=labels`: Automatically calculates the loss internally for classification.
        # - `loss`: The CrossEntropy loss between predicted logits and true labels.
        # - `logits`: Raw predictions before applying softmax.

        # Backward pass
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch}, Loss: {loss.item()}")

# Explanation:
# - `loss.backward()`: Computes gradients for each parameter.
# - `optimizer.step()`: Updates model parameters using computed gradients.
# - `loss.item()`: Retrieves the loss as a Python float for logging.

# Step 7: Evaluation
model.eval()  # Set model to evaluation mode

# Example new sentences
test_texts = ["I adore coding!", "This is the worst error."]
test_tokens = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")
test_input_ids = test_tokens["input_ids"]
test_attention_mask = test_tokens["attention_mask"]

# Predict
with torch.no_grad():
    predictions = model(test_input_ids, attention_mask=test_attention_mask).logits
    predicted_classes = torch.argmax(predictions, dim=1)

print("Predicted Sentiments:", predicted_classes.tolist())

# Explanation:
# - `torch.no_grad()`: Disables gradient calculation to save memory during inference.
# - `torch.argmax(predictions, dim=1)`: Converts raw logits into class predictions.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 0, Loss: 0.6770261526107788
Epoch: 1, Loss: 0.699068546295166
Predicted Sentiments: [1, 0]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformer import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset